<span style='color:#009999'> <span style='font-family:serif'> <font size="15"> **CMIP6** - Coupled Model Intercomparison Project Phase 6

<img src="img/Surface_Temperature.png" alt="drawing" width="750"/>    


<span style='color:#ff6666'><font size="5">**Additional Requirements**

- <font size="3"><span style='color:Black'> None.


 <span style='color:#ff6666'><font size="5"> **Objectives**
- <font size="3"><span style='color:Black'> To demonstrate remote access to CMIP data available through the <font size="3.5"><span style='color:#0066cc'>**Earth System Grid Federation** <font size="3.5"><span style='color:black'> [ESGF](https://aims2.llnl.gov/search/cmip6/) Portal.
- <font size="3"><span style='color:Black'> To access and subset remote data using the DAP2 Protocol.
- <font size="3"><span style='color:Black'> Understand the subtle differences between DAP2 and DAP4.
- <font size="3"><span style='color:Black'> To identify when an OPeNDAP server only implements DAP2.



<span style='color:#ff6666'><font size="5"> **Browsing Data:**

The <font size="3.5"><span style='color:#0066cc'>**Earth System Grid Federation** <font size="3.5"><span style='color:black'> [ESGF](https://aims2.llnl.gov/search/cmip6/) Contains a broad range of model output (e.g, CMIP3, CMIP5, [CMIP6](https://pcmdi.llnl.gov/CMIP6/), E3SM) from which you can obtain OPeNDAP URLs for data variables. 

The <font size="3.5"> To access the ESGF Node and browse data [click here](https://aims2.llnl.gov/search/cmip6/).

<img src="img/ESGF.png" alt="drawing" width="750"/>    



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from pydap.client import open_url
import cartopy.crs as ccrs

<span style='font-family:serif'> <font size="5.5"><span style='color:#0066cc'> **CMIP6 Access via OPeNDAP**

<font size="3.5">You can also directly inspect a THREDDS catalog for [CMIP6](https://crd-esgf-drc.ec.gc.ca/thredds/catalog/esgB_dataroot/AR6/CMIP6/catalog.html). For example, you can navigate to `CDRMIP/CCCma/CanESM5/esm-pi-cdr-pulse/r2i1p2f1/Eday/ts/gn/v20190429` and access [ts data](https://crd-esgf-drc.ec.gc.ca/thredds/dodsC/esgB_dataroot/AR6/CMIP6/CDRMIP/CCCma/CanESM5/esm-pi-cdr-pulse/r2i1p2f1/Eday/ts/gn/v20190429/ts_Eday_CanESM5_esm-pi-cdr-pulse_r2i1p2f1_gn_54510101-56501231.nc.html) via OPeNDAP DAP2 protocol.



In [ ]:
url = "https://crd-esgf-drc.ec.gc.ca/thredds/dodsC/esgB_dataroot/AR6/CMIP6/CDRMIP/CCCma/CanESM5/esm-pi-cdr-pulse/r2i1p2f1/Eday/ts/gn/v20190429/ts_Eday_CanESM5_esm-pi-cdr-pulse_r2i1p2f1_gn_54510101-56501231.nc"

<span style='font-family:serif'> <font size="5.5"><span style='color:#0066cc'> **Create dataset access via pydap**

- <font size="3.5"> By default `protocol='dap2'`, however the default behavior may change in the future.


In [ ]:
%%time
ds = open_url(url, protocol='dap2')

In [ ]:
ds.tree()

In [ ]:
print('Dataset memory user [GBs, uncompressed]: ', ds.nbytes/1e9)

<span style='font-family:serif'> <font size="5.5"><span style='color:#0066cc'> **Inspect single variable**



In [ ]:
ts = ds['ts']
ts

<span style='font-family:serif'> <font size="5.5"><span style='color:#0066cc'> **Grid Arrays**

-  <font size="3.5"> No longer implemented in `DAP4`. These carry copies of dimensions/coverage, and can be considered self-contained.
-  <font size="3.5"> Attempting to download into memory `ts` also downloads `time`, `lat`, `lon`.
-  <font size="3.5"> Attributes sit the `GridType` level. For example:

```python
ds['ts'].attributes
```
<font size="3.5"> and
```python
ds['ts']['ts'].attributes
```
<font size="3.5"> yield different results.


In [ ]:
def decode(variable) -> np.ndarray:
    """Decodes the variable BaseType according with atributes:
        _FillValue
        scale_factor

    Parameters:
        variable: BaseType (pydap model)
    """
    import pydap
    scale_factor = 1
    _Fillvalue = None

    if 'scale_factor' in variable.attributes:
        scale_factor = variable.scale_factor
    if '_FillValue' in variable.attributes:
        if isinstance(variable, pydap.model.GridType):
            data = np.where(variable.array.data == variable._FillValue, np.nan, variable.array.data) 
        elif isinstance(variable, pydap.model.BaseType):
            data = np.where(variable.data == variable._FillValue, np.nan, variable.data)    
    else:
        data = variable.data
    return scale_factor * data

In [ ]:
ts.tree()

<span style='color:#ff6666'><font size="5"> **Exercise**

<font size="3.5"><span style='color:black'> Make a surface map of a variable (say `ts` in the example), for `time=0`. You can do that in two ways:


- <font size="3.5"><span style='color:black'> Download the Array `ts` into memory from the original URL via pydap (`GridType` array)
- <font size="3.5"><span style='color:black'> Append a Constraint Expression (CE) to the original `dataURL` only download the data you want. You can do this interactively in the DAP response form of the dataset. Simply paste original url `<dataURL>+'.html'` onto a browser to view the DAP response form, and then there select only a single time index value.



<font size="3.5"><span style='color:black'> **NOTE**: When making a plot, check for missing values, scale factors, units.





<span style='font-family:serif'> <font size="5.5"><span style='color:#0066cc'> **pydap approach:**

- <font size="3.5"> **NOTE** Some Data providers specify a limit to how much data can be downloaded at once. This upper value limit can be configured within any OPeNDAP server.


In [ ]:
%%time
# Attempting to download the entire GridType triggers an error on the server side.
GTS = ds['ts'][:]
GTS

In [ ]:
ds['ts'].shape

In [ ]:
%%time
# download the entire GridType, single snapshot
GTS = ds['ts'][0, :, :]
GTS

In [ ]:
GTS.attributes

In [ ]:
GTS.shape

In [ ]:
len(GTS.data), type(GTS.data)

<span style='color:#ff6666'><font size="5"> **Question**:

<font size="3.5"> Why

```python
# why are the two different?
len(GTS.data) != GTS.shape ?? 
```
                                              
<font size="3.5"> Where is the data? Try:

```python
[GTS.data[i].shape for i in range(len(GTS.data))]
```


In [ ]:
%%time
# download the only Array, single snapshot
TS = ds['ts']['ts'][0, :, :]
TS

In [ ]:
TS.attributes

In [ ]:
Lon, Lat = np.meshgrid(GTS['lon'].data, GTS['lat'].data)

In [ ]:
plt.figure(figsize=(15, 5))
ax = plt.axes(projection=ccrs.Mollweide())
ax.set_global()
ax.coastlines()
ax.contourf(Lon, Lat, np.squeeze(decode(GTS)), 200, transform=ccrs.PlateCarree(), cmap='jet')
plt.show()

<span style='color:#ff6666'><font size="5"> **Exercise**:

- <font size="3.5"> Browse through the THREDDS catalog, an explore another flow variable. If you don't want to explore, you can inspect the Air Temperature variable with dataURL listed below:

```python
url = "https://crd-esgf-drc.ec.gc.ca/thredds/dodsC/esgB_dataroot/AR6/CMIP6/CDRMIP/CCCma/CanESM5/1pctCO2-cdr/r1i1p2f1/Amon/ta/gn/v20190429/ta_Amon_CanESM5_1pctCO2-cdr_r1i1p2f1_gn_199101-219012.nc"
```
